In [1]:
import pandas as pd
import numpy as np

import seaborn as sns 
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, confusion_matrix
import re
from tqdm import tqdm_notebook
from pathlib import Path
from multiprocessing import Pool, cpu_count

import json
import torch
import pprint
import numpy as np
import itertools
import functools
import os

from matplotlib import pyplot as plt
%matplotlib inline


import requests
from lxml import html
from bs4 import BeautifulSoup

In [3]:
r = requests.get("https://archive.org/download/stackexchange")
soup = BeautifulSoup(r.content, "html.parser")


In [9]:
listing_table = soup.find('table', class_='directory-listing-table')

In [12]:
links = listing_table.findAll('a', href=True)

In [27]:
link_list = '\n'.join(['https://archive.org/download/stackexchange/' + l['href'] for l in links[1:]])

In [30]:
!mkdir /storage/monty/quest_challenge/stackexchange_dump

In [32]:
!df /storage/monty/quest_challenge/stackexchange_dump -h

Filesystem      Size  Used Avail Use% Mounted on
/dev/md0        2.5T  772G  1.6T  33% /storage


In [33]:
with open('/storage/monty/quest_challenge/stackexchange_dump/link_list.txt','w') as f:
    f.write(link_list)

In [35]:
len(links)

362

In [6]:
!ls /storage/monty/quest_challenge/stackexchange_dump/anime.stackexchange.com -alh

total 149M
drwx--S---   2 jupyter-monty jupyter-monty 4.0K Dec 16 14:31 .
drwxr-sr-x 348 jupyter-monty jupyter-monty  36K Dec 16 15:04 ..
-rw-r--r--   1 jupyter-monty jupyter-monty 5.5M Dec  2 04:12 Badges.xml
-rw-r--r--   1 jupyter-monty jupyter-monty 9.2M Dec  2 04:12 Comments.xml
-rw-r--r--   1 jupyter-monty jupyter-monty  71M Dec  2 04:12 PostHistory.xml
-rw-r--r--   1 jupyter-monty jupyter-monty 350K Dec  2 04:12 PostLinks.xml
-rw-r--r--   1 jupyter-monty jupyter-monty  36M Dec  2 04:12 Posts.xml
-rw-r--r--   1 jupyter-monty jupyter-monty 111K Dec  2 04:12 Tags.xml
-rw-r--r--   1 jupyter-monty jupyter-monty  11M Dec  2 04:12 Users.xml
-rw-r--r--   1 jupyter-monty jupyter-monty  17M Dec  2 04:12 Votes.xml


In [40]:
!ls /storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com   

Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml


In [41]:
!ls /storage/monty/quest_challenge/stackexchange_dump/3dprinting.stackexchange.com 

Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml


In [45]:
!cat /storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com/Posts.xml
























































































































































































































































































































































































































































</posts>

In [8]:
!ls /storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com/

Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml


In [9]:
!cat /storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com/Badges.xml

In [10]:
!cat /storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com/Comments.xml









































































































































































































































































































































































































































































































































































































































</comments>

In [13]:
!cat /storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com/Posts.xml
























































































































































































































































































































































































































































</posts>

In [15]:
!cat /storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com/Users.xml










































































































































































































































































































































































































































































































































































































































































































































</users>

In [18]:
!ls /storage/monty/quest_challenge/stackexchange_dump/*com

/storage/monty/quest_challenge/stackexchange_dump/3dprinting.meta.stackexchange.com:
Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml

/storage/monty/quest_challenge/stackexchange_dump/3dprinting.stackexchange.com:
Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml

/storage/monty/quest_challenge/stackexchange_dump/academia.meta.stackexchange.com:
Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml

/storage/monty/quest_challenge/stackexchange_dump/academia.stackexchange.com:
Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml

/storage/monty/quest_challenge/stackexchange_dump/ai.meta.stackexchange.com:
Badges.xml    PostHistory.xml  Posts.xml  Users.xml
Comments.xml  PostLinks.xml    Tags.xml   Votes.xml

/storage/monty/quest_challenge/stackexchange_dump/ai.stackexchange.com:


In [58]:
from pathlib import Path
stackexchange_dir = Path('/storage/monty/quest_challenge/stackexchange_dump/3dprinting.stackexchange.com/')

In [59]:
import xml.etree.cElementTree as et
import pandas as pd


def xml_to_pandas(root, columns, row_name='row'):
    df = None
    try:

        rows = root.findall('.//{}'.format(row_name))

        xml_data = [[row.get(c) for c in columns] for row in rows]  # NESTED LIST

        df = pd.DataFrame(xml_data, columns=columns)
    except Exception as e:
        print('[xml_to_pandas] Exception: {}.'.format(e))

    return df

In [121]:
stackexchange_dir

PosixPath('/storage/monty/quest_challenge/stackexchange_dump/3dprinting.stackexchange.com')

In [128]:
def parse_xml_dump(pathes):
    stackexchange_dir, output_dir = pathes
    
    path = stackexchange_dir / 'Users.xml'
    columns =['Id', 'Reputation', 'DisplayName']

    root = et.parse(path)
    user_df = xml_to_pandas(root, columns, row_name)
    user_df = user_df.rename(columns ={
        'Reputation':'user_reputation',
        'DisplayName':'username',
        'Id':'OwnerUserId',
    })
    
    path = stackexchange_dir / 'Posts.xml'
    columns = [
        'AcceptedAnswerId',
        'AnswerCount',
        'Body',
        'ClosedDate',
        'CommentCount',
        'CreationDate',
        'FavoriteCount',
        'Id',
        'LastActivityDate',
        'OwnerUserId',
        'ParentId',
        'PostTypeId',
        'Score',
        'Title',
        'ViewCount']

    root = et.parse(path)
    posts_df = xml_to_pandas(root, columns)

    question_columns = ['Id',
     'CreationDate',
     'Score',
     'ViewCount',
     'Body',
     'OwnerUserId',
     'LastActivityDate',
     'Title',
     'AnswerCount',
     'CommentCount',
     'FavoriteCount',
     'AcceptedAnswerId',
     'ClosedDate',]

    answer_columns =['Id',
     'CreationDate',
     'Score',
     'Body',
     'OwnerUserId',
     'LastActivityDate',
     'CommentCount',
     'ParentId']

    question_df = posts_df[posts_df['PostTypeId']== '1'][question_columns]
    answer_df = posts_df[posts_df['PostTypeId']== '2'][answer_columns]

    answer_df = answer_df.merge(user_df, on='OwnerUserId')
    question_df = question_df.merge(user_df, on='OwnerUserId')
    
    answer_df.to_csv(output_dir / 'answers.tsv', sep='\t', index=False)
    question_df.to_csv(output_dir / 'questions.tsv', sep='\t', index=False)
    
#     return question_df, answer_df

In [141]:
output_dir = Path('/storage/monty/quest_challenge/stackexchange_parsed')

dumps = list(Path('/storage/monty/quest_challenge/stackexchange_dump/').glob('*com'))
dumps = [path for path in dumps if '.meta' not in path.name]

outputs = [output_dir / path.name for path in dumps]
for path in outputs:
    if not path.exists():
        os.makedirs(str(path))

In [150]:
with Pool(cpu_count()) as pool:
    list(tqdm_notebook(pool.imap(parse_xml_dump, zip(dumps, outputs)), total=len(dumps)))

/home/jupyter-monty/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [153]:
stackoverflow_output = Path('/storage/monty/quest_challenge/stackexchange_parsed/stackoverflow/')
stackoverflow_dump = Path('/storage/monty/quest_challenge/stackexchange_dump/stackoverflow/')

In [ ]:
%%time
parse_xml_dump((stackoverflow_dump, stackoverflow_output))

In [118]:
answer_df

,Id,CreationDate,Score,Body,OwnerUserId,LastActivityDate,CommentCount,ParentId,user_reputation,username
0,9,2016-01-12T18:58:04.410,17,<p>Almost all 3D printers have issues that cou...,16,2016-01-12T18:58:04.410,2,2,1783,Adam Davis
1,39,2016-01-12T20:00:22.177,15,<p>The files used to print these objects are d...,16,2016-01-12T20:00:22.177,0,33,1783,Adam Davis
2,43,2016-01-12T20:05:22.530,6,"<p>In theory, making filament is easy. You ta...",16,2019-01-18T15:13:48.737,2,38,1783,Adam Davis
3,53,2016-01-12T20:24:10.350,4,<blockquote>\n <p>parts ... I don't want to ....,16,2016-01-13T12:37:29.580,0,14,1783,Adam Davis
4,57,2016-01-12T20:30:29.773,6,<p>Using negative pressure ventilation and a s...,16,2016-01-12T20:30:29.773,0,49,1783,Adam Davis
...,...,...,...,...,...,...,...,...,...,...
4903,11425,2019-11-23T22:42:01.473,2,<p>Long story short: <strong>Try to replace th...,19228,2019-11-24T08:06:09.650,0,11424,151,sharpener
4904,11426,2019-11-24T16:58:09.213,1,<p>This I hope will help: </p>\n\n<ol>\n<li><p...,19235,2019-11-24T17:50:57.497,1,8474,11,George S.
4905,11428,2019-11-25T10:50:23.540,2,<ul>\n<li>Tune -> Bed Z</li>\n</ul>\n\n<p><a h...,17202,2019-11-25T10:50:23.540,1,11422,121,mikezter
4906,11430,2019-11-25T13:15:13.563,1,"<p>As Mark said, Marlin supports a number of m...",19239,2019-11-27T08:41:16.547,0,7676,11,Marcelo Nicolás Obachi


In [8]:
pd.read_csv('/storage/monty/quest_challenge/stackexchange_parsed/academia.stackexchange.com/answers.tsv', sep='\t')

,Id,CreationDate,Score,Body,OwnerUserId,LastActivityDate,CommentCount,ParentId,user_reputation,username
0,6,2012-02-14T20:30:29.917,28,<p>If your institution has a subscription to J...,18,2017-01-11T18:37:45.013,2,3,1054,Chang
1,110,2012-02-15T15:58:25.420,6,"<p>If you are interested in a teaching job, my...",18,2012-02-15T16:33:54.190,0,53,1054,Chang
2,9,2012-02-14T20:39:18.063,31,<p>The h-index is a measure of the impact of s...,12,2012-02-14T20:39:18.063,3,5,5044,Lars Kotthoff
3,11,2012-02-14T20:40:32.307,8,"<p>There is a list at <a href=""http://www.scie...",12,2012-02-14T20:40:32.307,0,3,5044,Lars Kotthoff
4,22,2012-02-14T20:54:44.607,3,<p>Depending on which country you are talking ...,12,2012-02-14T20:54:44.607,0,18,5044,Lars Kotthoff
...,...,...,...,...,...,...,...,...,...,...
69642,140893,2019-11-30T17:27:26.867,1,"<p>Within reason, it's fine. And probably sho...",116854,2019-11-30T18:02:55.197,0,140855,31,guest
69643,140894,2019-11-30T17:40:11.703,2,<p>In most experimental fields (to which I wou...,116854,2019-11-30T22:53:46.870,0,140882,31,guest
69644,140895,2019-11-30T18:02:32.933,1,<p>You can certainly mention professor's names...,116855,2019-11-30T18:02:32.933,0,15016,11,S.R
69645,140900,2019-11-30T18:49:31.870,-2,<p>Depending on where you are and who is your ...,112841,2019-11-30T18:49:31.870,3,140877,113,fraxinus


In [33]:
path = stackexchange_dir / 'Comments.xml'
columns =['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId']

root = et.parse(path)
df = xml_to_pandas(root, columns, row_name)
df

,Id,PostId,Score,Text,CreationDate,UserId
0,1,1,7,I am in the same position. I know very little...,2016-01-12T19:31:31.027,23
1,2,3,0,"Sorry, I implied that it should only be mandat...",2016-01-12T20:12:13.977,10
2,3,2,0,"If printer tags would be mandatory, I also thi...",2016-01-12T20:43:30.493,43
3,4,2,0,"@EricJohnson true, I guess I over-thought this...",2016-01-12T20:46:31.583,10
4,5,9,2,"Agree. I think at this stage of development, ...",2016-01-12T20:49:28.737,16
...,...,...,...,...,...,...
609,697,443,0,Note that Question 2 likely won't get answers....,2019-11-11T13:24:51.537,8884
610,698,396,0,I have changed the language within the templat...,2019-11-11T23:06:41.617,11242
611,699,396,0,"@Pᴀᴜʟsᴛᴇʀ2 - good point, well spotted... it'll...",2019-11-12T19:12:16.167,4762
612,700,396,0,@Greenonline - AH! I should have known ... gla...,2019-11-12T21:38:59.010,11242


In [31]:
# str(stackexchange_dir / 'Users.xml')
!cat /storage/monty/quest_challenge/stackexchange_dump/3dprinting.stackexchange.com/Comments.xml

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [32]:
'Id PostId Score Text CreationDate UserId'.split()


['Id', 'PostId', 'Score', 'Text', 'CreationDate', 'UserId']